In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
df = pd.read_csv('HRDB_DATA.csv')

# EDA 
- Identify missing values and handle them appropriately.
- Perform summary statistics and visualize data distributions.
- Identify potential correlations between features that may contribute to
attrition.

In [ ]:
df.shape

In [ ]:
df.info()
#אין ערכים חסרים בטבלה!!

In [ ]:
#בדיקה כמה ערכים "יוניקים" יש בטבלה
df.nunique(axis=0)

In [ ]:
#סטטיסטיקה תיאורית
df.describe()

In [ ]:
joined_table = pd.read_csv("HRDB_DATA.csv")
joined_table

In [ ]:
joined_table

# Correlation Matrix
## Detailed Correlation Table
### Shows all corr values

In [ ]:
correlation_matrix = joined_table.select_dtypes(include=['number']).corr()
correlation_matrix.round(2)

## Heatmap
### Filteres |corr|>0.5

In [ ]:
corr = joined_table.corr(numeric_only=True)
# Filter correlations: keep only corr > 0.5 or < -0.5 (diagonal will always be 1)
filtered_corr = corr[(corr > 0.5) | (corr < -0.5)]
plt.figure(figsize=(12,8))
sns.heatmap(filtered_corr, annot=True, cmap='coolwarm', center=0, mask=filtered_corr.isnull(), cbar_kws={'label': 'Correlation'})
plt.title('Correlation Heatmap (corr < -0.5 or corr > 0.5)')
plt.show()

## Top Correlations
### Shows pairs of features with highest corr values (desc order)

In [ ]:
# Get the correlation matrix, keeping only upper triangle (without diagonal)
corr_matrix = joined_table.corr(numeric_only=True)
corr_pairs = (
    corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    .stack()
    .reset_index()
)
corr_pairs.columns = ["Feature 1", "Feature 2", "Correlation"]

# Filter by correlation magnitude > 0.5
strong_corr = corr_pairs[(corr_pairs["Correlation"].abs() < -0.5)]

# Sort descending by absolute correlation value
strong_corr = strong_corr.reindex(
    strong_corr["Correlation"].abs().sort_values(ascending=False).index
)

# Filter by correlation magnitude > 0.5 or < -0.5
strong_corr = corr_pairs[(corr_pairs["Correlation"].abs() > 0.5)]
strong_corr.round(2)


In [ ]:
# If Attrition is encoded as 'Yes'/'No', convert it to 1/0:
if joined_table['Attrition'].dtype == object:
    joined_table['Attrition'] = joined_table['Attrition'].map({'Yes': 1, 'No': 0})

# Create age bins
age_bins = [18, 25, 35, 45, 55, 65]
age_labels = ['18-24', '25-34', '35-44', '45-54', '55+']
joined_table['Age Group'] = pd.cut(joined_table['Age'], bins=age_bins, labels=age_labels, right=False)

# Attrition rate by age group
attrition_by_age_group = joined_table.groupby('Age Group')['Attrition'].mean()
print("\nAttrition Rate by Age Group:")
print(attrition_by_age_group.apply(lambda x: f"{x:.2%}"))

plt.figure(figsize=(6,4))
attrition_by_age_group.plot(kind='bar', color='green')
plt.ylabel("Attrition Rate")
plt.title("Attrition Rate by Age Group")
plt.ylim(0, 1)
plt.xticks(rotation=0)
plt.show()


In [ ]:
# Analyze attrition rate by department
attrition_by_dept = joined_table.groupby("Department")["Attrition"].mean().sort_values(ascending=False)
print("\nAttrition Rate by Department:")
print(attrition_by_dept.apply(lambda x: f"{x:.2%}"))
plt.figure(figsize=(8,4))
attrition_by_dept.plot(kind='bar', color='skyblue')
plt.ylabel("Attrition Rate")
plt.title("Attrition Rate by Department")
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.show()




In [ ]:

# Analyze attrition rate by gender
attrition_by_gender = joined_table.groupby("Gender")["Attrition"].mean().sort_values(ascending=False)
print("\nAttrition Rate by Gender:")
print(attrition_by_gender.apply(lambda x: f"{x:.2%}"))

plt.figure(figsize=(5,4))
attrition_by_gender.plot(kind='bar', color='orange')
plt.ylabel("Attrition Rate")
plt.title("Attrition Rate by Gender")
plt.ylim(0, 1)
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Comparison of Job Levels 1, 2, 3, 4, 5 on Key Specifications and Attributes

# Filter data for job levels 1, 2, 3, 4, and 5
selected_levels = [1, 2, 3, 4, 5]
levels_df = joined_table[joined_table["JobLevel"].isin(selected_levels)].copy()

# Set the display order for Job Levels
levels_df["JobLevel"] = levels_df["JobLevel"].astype("category")
levels_df["JobLevel"] = levels_df["JobLevel"].cat.set_categories(selected_levels, ordered=True)

# 1. Summary Table: Mean of Key Numeric Attributes
attributes_to_summarize = [
    "HourlyRate", "MonthlyIncome", "YearsAtCompany", 
    "YearsSinceLastPromotion", "TotalWorkingYears", "Attrition"
]
summary_table = levels_df.groupby("JobLevel")[attributes_to_summarize].mean().round(2)
summary_table.rename(columns={"Attrition": "Attrition Rate"}, inplace=True)
summary_table["Attrition Rate"] = summary_table["Attrition Rate"].apply(lambda x: f"{x:.2%}")

print("Summary Statistics for Job Levels 1, 2, 3, 4, 5:")
display(summary_table)

# 2. Bar Plots: Compare Means of Key Numeric Attributes
import matplotlib.pyplot as plt

# Plotting
fig, axs = plt.subplots(3, 2, figsize=(12, 12))
ax = axs.flatten()

for idx, attr in enumerate(attributes_to_summarize):
    # Special formatting for Attrition (rate, not average raw value)
    if attr == "Attrition":
        data = levels_df.groupby("JobLevel")[attr].mean()
        ax[idx].bar(data.index.astype(str), data.values, color='coral')
        ax[idx].set_ylabel("Attrition Rate")
        ax[idx].set_title("Attrition Rate by Job Level")
        ax[idx].set_ylim(0, 1)
        ax[idx].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0f}%'.format(100 * y)))
    else:
        data = levels_df.groupby("JobLevel")[attr].mean()
        ax[idx].bar(data.index.astype(str), data.values, color='skyblue')
        ax[idx].set_ylabel(attr)
        ax[idx].set_title(f"{attr} by Job Level")
    ax[idx].set_xlabel("Job Level")

plt.tight_layout()
plt.show()

# 3. Categorical Attribute Distribution: Job Role by Level
ct_jobrole = pd.crosstab(levels_df["JobRole"], levels_df["JobLevel"])

# Normalize by columns to get the fraction within each job level
ct_jobrole_pct = ct_jobrole.div(ct_jobrole.sum(axis=0), axis=1).round(3)